In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import(Embedding, LSTM, Dense, Input, Layer , Dropout,GlobalAveragePooling1D)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import (Embedding, LSTM, Dense, Input, Layer, Dropout,
                                     GlobalAveragePooling1D)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

max_features = 2000
max_len = 200
emdediding_dim = 123

# charge dataset
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


# define attention layer

class Attenttion(Layer):
    def __init__(self):
        super(Attenttion, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer="random_normal",
                                 trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],),
                                 initializer="zeros",
                                 trainable=True)
        self.u = self.add_weight(shape=(input_shape[-1], 1),
                                 initializer="random_normal",
                                 trainable=True)

        super(Attenttion, self).build(input_shape)

    def call(self, inputs):
        # calculate score attention
        score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b) # Changed 'input' to 'inputs'
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
        # calcule du contex pondere
        context_vector = tf.reduce_sum(inputs * attention_weights, axis=1)
        return context_vector, attention_weights


# constract model with LSTM and attention

def build_model():
    inputs = Input(shape=(max_len,))
    x = Embedding(max_features, emdediding_dim)(inputs)
    x = LSTM(128, return_sequences=True)(x)  # output of Attention sequence
    context_vector, attention_weights = Attenttion()(x)
    x = Dropout(0.5)(context_vector)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


# compile model
model = build_model()
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

history = model.fit(X_train, Y_train, batch_size=64, epochs=5,
                    validation_data=(X_test, Y_test))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 200, 123)            │         246,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 200, 128)            │         129,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attenttion_1 (Attenttion)            │ [(None, 128), (None, 200,   │          16,640 │
│                                      │ 1)]                         │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 391,793 (1.49 MB)

 Trainable params: 391,793 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.6760 - loss: 0.5609 - val_accuracy: 0.8652 - val_loss: 0.3131
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.8781 - loss: 0.2985 - val_accuracy: 0.8756 - val_loss: 0.2879
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.8988 - loss: 0.2532 - val_accuracy: 0.8777 - val_loss: 0.2853
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.9075 - loss: 0.2308 - val_accuracy: 0.8796 - val_loss: 0.2803
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9189 - loss: 0.2093 - val_accuracy: 0.8810 - val_loss: 0.2864


In [ ]:


#Evaluate model
test_loss,test_acc=model.evaluate(X_test,Y_test)
print("Test Accuracy:",test_acc)

#visuliaze attention weights
def visulize_attention(input_sequence,word_index):
  reverse_word_index={v:k for k,v in word_index.items()}
  words=[reverse_word_index.get(i,'?') for i in input_sequence]

  input_sequence= pad_sequences([input_sequence],maxlen=max_len)
  model_selection= Model(inputs=model.input,outputs=model.get_layer('attenttion_1').output)
  contex_vector,attention_weights=model_selection.predict(input_sequence)
  attention_weights=attention_weights.squeeze()
  #print word and their words
  for word , weight in zip(words,attention_weights):
    print(f"{word}:{weight}")

sample_index=0
visulize_attention(X_train[sample_index],word_index=imdb.get_word_index())





782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8799 - loss: 0.2885
Test Accuracy: 0.8810399770736694
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
to:0.0025466508232057095
have:0.0030701318755745888
after:0.0012352573685348034
out:0.0017693248810246587
atmosphere:0.0022786774206906557
never:0.0024462032597512007
more:0.0019163611577823758
room:0.007899800315499306
and:0.0028005572967231274
it:0.002269358141347766
so:0.0026202485896646976
heart:0.012132992967963219
shows:0.0034025325439870358
to:0.0026308640372008085
years:0.0017241091700270772
of:0.0011503141140565276
every:0.002774552907794714
never:0.00269005517475307
going:0.008184238336980343
and:0.002833239734172821
help:0.0016428310191258788
moments:0.0012589169200509787
or:0.0008862106478773057
of:0.0007711134967394173
every:0.0018109593074768782
and:0.0012873192317783833
visual:0.010828280821442604
movie:0.004637171979993582
except:0.005061938893049955
her:0.0042901416309177876
was:0.003694161307066679
several:0.00975620280951